In [ ]:
import pandas as pd
import numpy as np
import matplotlib
matplotlib.use('nbagg')
import matplotlib.pyplot as plt
import re
import json

In [ ]:
df = pd.read_json('data/craigslist.json')
df.head(10)

In [ ]:
def fix_address(address):
    if address != None:
        address = address.lower()
        near_issue = address.find("near")
        apt_issue = address.find("apt")
        if near_issue != -1:
            address = address[0:near_issue-1]
        if apt_issue != -1:
            address = address[0:apt_issue-1]
        return address
    
# and then call apply on the series to apply the function to each element in the series
df['address'] = df['address'].apply(fix_address)
df.head(10)

In [ ]:
def fix_none(address):
    if address != None:
        return True
    else:
        return False

clean_df = df[df['address'].apply(fix_none)]
    
df['address'] = df['address'].apply(fix_address)
clean_df.head(10)

In [ ]:
usable_df = clean_df[['address','price']]
usable_df.head(10)